# Value Mapping

In this notebook, we'll explore value mapping in depth. Value mapping is a powerful feature of the Elasticsearch Query Toolkit that allows you to define relationships between directive names, fields, and values in a structured, reusable way.

## Introduction to Value Mapping

Value mapping solves several common problems in query construction:

1. **Field-Value Association**: Mapping directive names to specific fields and values
2. **Dynamic Value Resolution**: Resolving values from runtime parameters
3. **Value Transformation**: Transforming values before they're used in queries
4. **Reusability**: Defining field-value mappings once and reusing them across multiple queries

The toolkit provides the `DirectiveValueMapper` class for defining these mappings.

## Setup

Let's import the necessary modules:

In [ ]:
import json
from elastictoolkit.queryutils.builder.directivevaluemapper import DirectiveValueMapper
from elastictoolkit.queryutils.types import FieldValue, NestedField
from elastictoolkit.queryutils.builder.directiveengine import DirectiveEngine
from elastictoolkit.queryutils.builder.matchdirective import (
    ConstMatchDirective,
    TextMatchDirective,
    RangeMatchDirective
)
from elastictoolkit.queryutils.consts import FieldMatchType

# Helper function to print queries as formatted JSON
def print_query(engine):
    query = engine.to_dsl()
    print(json.dumps(query.to_query(), indent=2))

## Creating a Basic Value Mapper

Let's start by creating a basic value mapper for a product search application:

In [ ]:
class ProductValueMapper(DirectiveValueMapper):
    # Static field-value mapping
    in_stock = FieldValue(
        fields=["in_stock"],
        values_list=[True]
    )
    
    # Dynamic field-value mapping
    category = FieldValue(
        fields=["category"],
        values_list=["match_params.category"]
    )
    
    # Multiple fields mapping
    search = FieldValue(
        fields=["name", "description"],
        values_list=["match_params.search_text"]
    )
    
    # Range mapping
    price_range = FieldValue(
        fields=["price"],
        values_map={
            "gte": "match_params.min_price",
            "lte": "match_params.max_price"
        }
    )

# Create a value mapper instance
value_mapper = ProductValueMapper()

# Get field-value mapping for a specific directive
attr_field_value = value_mapper.get_field_value("category")
print(f"Fields: {attr_field_value.fields}")
print(f"Values List: {attr_field_value.values_list}")
print(f"Values Map: {attr_field_value.values_map}")

In this example, we've defined a `ProductValueMapper` with several field-value mappings:

1. `in_stock`: A static mapping that always maps to the field "in_stock" with the value `True`
2. `category`: A dynamic mapping that maps to the field "category" with a value from `match_params.category`
3. `search`: A mapping to multiple fields ("name" and "description") with a value from `match_params.search_text`
4. `price_range`: A range mapping with values from `match_params.min_price` and `match_params.max_price`

The `get_field_value_mapping` method returns the fields, values list, and values map for a specific directive.

## Using Value Mappers with Engines

Let's see how to use our value mapper with a directive engine:

In [ ]:
class ProductSearchEngine(DirectiveEngine):
    # Define directives as class attributes
    category = ConstMatchDirective(rule=FieldMatchType.ANY)
    in_stock = ConstMatchDirective(rule=FieldMatchType.ANY)
    search = TextMatchDirective(rule=FieldMatchType.ANY)
    price_range = RangeMatchDirective()
    
    class Config:
        value_mapper = ProductValueMapper()

# Create an engine instance with match parameters
engine = ProductSearchEngine().set_match_params(
    {
        "category": "electronics",
        "search_text": "wireless headphones",
        "min_price": 50,
        "max_price": 200
    }
)

# Generate the query
print_query(engine)

When we create an engine instance with match parameters, the engine uses the value mapper to resolve the fields and values for each directive. The value mapper looks up the field-value mapping for each directive name and applies the match parameters to resolve dynamic values.

## Dynamic Value Resolution

One of the most powerful features of value mapping is dynamic value resolution. This allows you to define values that are resolved at runtime based on match parameters or other dynamic sources.

### Basic Dynamic Resolution

The simplest form of dynamic resolution is referencing match parameters:

In [ ]:
class DynamicValueMapper(DirectiveValueMapper):
    category = FieldValue(
        fields=["category"],
        values_list=["match_params.category"]  # Reference to match parameter
    )
    price_range = FieldValue(
        fields=["price"],
        values_map={
            "gte": "match_params.min_price",  # Reference to match parameter
            "lte": "match_params.max_price"   # Reference to match parameter
        }
    )

# Create a value mapper instance
value_mapper = DynamicValueMapper()

# Create match parameters
match_params = {
    "category": "electronics",
    "min_price": 50,
    "max_price": 200
}

# Get field-value mapping for category
attr_field_value = value_mapper.get_field_value("category")
print("Category Mapping:")
print(f"  Fields: {attr_field_value.fields}")
print(f"  Values List: {attr_field_value.values_list}")
print()

# Resolve values using a value parser
from elastictoolkit.queryutils.builder.helpers.valueparser import RuntimeValueParser

parser = RuntimeValueParser(data=match_params, prefix="match_params")
resolved_values = parser.parse(attr_field_value.values_list)
print(f"Resolved Values: {resolved_values}")

### Using Callables for Dynamic Values

You can also use callables (functions) for more complex value resolution:

In [ ]:
# Resolve values using a value parser
from elastictoolkit.queryutils.builder.helpers.valueparser import RuntimeValueParser

def normalize_category(data):
    """Convert category to lowercase and remove spaces"""
    category = data.get("category", "")
    return category.lower().replace(" ", "_")

class CallableValueMapper(DirectiveValueMapper):
    category = FieldValue(
        fields=["category"],
        values_list=[normalize_category]  # Callable function
    )

# Create a value mapper instance
value_mapper = CallableValueMapper()

# Create match parameters
match_params = {
    "category": "Home & Garden",
    "min_price": 50,
    "max_price": 200
}

# Get field-value mapping for category
attr_field_value = value_mapper.get_field_value("category")
print("Category Mapping:")
print(f"  Fields: {attr_field_value.fields}")
print(f"  Values List: {attr_field_value.values_list}")
print()


parser = RuntimeValueParser(data=match_params, prefix="match_params")
resolved_values = parser.parse(attr_field_value.values_list)
print(f"Resolved Values: {resolved_values}")

In [ ]:
def get_price_min(data):
    """Get min price with default value"""
    min_price = data.get("min_price", 0)
    return min_price

def get_price_max(data):
    """Get max price with default value"""
    max_price = data.get("max_price", 1000)
    return max_price

class CallableValueMapper(DirectiveValueMapper):
    price_range = FieldValue(
        fields=["price"],
        values_map={"gte": get_price_min, "lte": get_price_max}  # Callable function
    )

# Create a value mapper instance
value_mapper = CallableValueMapper()

# Create match parameters
match_params = {
    "category": "Home & Garden",
    "min_price": 50,
    "max_price": 200
}

parser = RuntimeValueParser(data=match_params, prefix="match_params")

# Get field-value mapping for price range
attr_field_value = value_mapper.get_field_value("price_range")
print("Price Range Mapping:")
print(f"  Fields: {attr_field_value.fields}")
print(f"  Values Map: {attr_field_value.values_map}")
print()

# Resolve values
resolved_values_map = parser.parse(attr_field_value.values_map)
print(f"Resolved Values Map: {resolved_values_map}")

### Unpacking Values

Sometimes you need to unpack a list of values into the query. The toolkit supports this with the `*` prefix for string values and the `unpack` attribute for callables:

In [ ]:
# Resolve values using a value parser
from elastictoolkit.queryutils.builder.helpers.valueparser import RuntimeValueParser
from elastictoolkit.queryutils.builder.helpers.valuetransformer import ValueTransformer

def get_tags(data):
    """Get tags from data"""
    return data.get("tags", [])

class UnpackingValueMapper(DirectiveValueMapper):
    tags = FieldValue(
        fields=["tags"],
        values_list=["*match_params.tags"]  # Unpack this list
    )
    
    tags_callable = FieldValue(
        fields=["category"],
        values_list=[ValueTransformer.unpacked(get_tags)]  # Will be unpacked because func was wrapped with `ValueTransformer.unpacked`
    )

# Create a value mapper instance
value_mapper = UnpackingValueMapper()

# Create match parameters with lists
match_params = {
    "tags": ["wireless", "bluetooth", "headphones"],
    "categories": ["electronics", "audio"]
}

# Get field-value mapping for tags
attr_field_value = value_mapper.get_field_value("tags")
print("Tags Mapping:")
print(f"  Fields: {attr_field_value.fields}")
print(f"  Values List: {attr_field_value.values_list}")
print()


parser = RuntimeValueParser(data=match_params, prefix="match_params")
resolved_values = parser.parse(attr_field_value.values_list)
print(f"Resolved Values: {resolved_values}")
print()

# Get field-value mapping for categories with callable
attr_field_value = value_mapper.get_field_value("tags_callable")
print("Tags[callable] Mapping:")
print(f"  Fields: {attr_field_value.fields}")
print(f"  Values List: {attr_field_value.values_list}")
print()

resolved_values = parser.parse(attr_field_value.values_list)
print(f"Resolved Values: {resolved_values}")

In [ ]:
# Create match parameters with lists
match_params = {
    "tags": ["wireless", "bluetooth", "headphones"],
    "categories": ["electronics", "audio"]
}

parser = RuntimeValueParser(data=match_params, prefix="match_params")

# Demonstrate unpacking with a callable
def get_categories(data):
    return data.get("categories", [])

# Create a list with callable but not unpacked
test_list = ["static_value", get_categories]
resolved_list = parser.parse(test_list)
print(f"Resolved List without Unpacked Callable: {resolved_list} # List inside a list - this might not be the requirement")

# Create a list with the unpacked callable
test_list_unpacked = ["static_value", ValueTransformer.unpacked(get_categories)]
resolved_list_unpacked = parser.parse(test_list_unpacked)
print(f"Resolved List with Unpacked Callable: {resolved_list_unpacked}      # The list is unpacked and extended to original list")
print()

In these examples:

1. **Basic Dynamic Resolution**: We reference match parameters using the `match_params.` prefix
2. **Callable Resolution**: We use functions to transform or compute values dynamically
3. **Unpacking Values**: 
   - For strings, we use the `*` prefix to unpack lists (e.g., `*match_params.tags`)
   - For callables, we set the `unpack` attribute to `True` or use `ValueTransformer.unpacked()`

This powerful combination allows you to handle complex value resolution scenarios while keeping your code clean and maintainable.

## Working with Nested Fields

Value mappers also support Elasticsearch's nested fields through the `NestedField` class:

In [ ]:
class NestedValueMapper(DirectiveValueMapper):
    # Nested field mapping
    reviews = FieldValue(
        fields=[
            NestedField(nested_path="reviews", field_name="reviews.rating")
        ],
        values_list=["match_params.min_rating"]
    )
    
    # Multiple nested fields
    variants = FieldValue(
        fields=[
            NestedField(nested_path="variants", field_name="variants.color"),
            NestedField(nested_path="variants", field_name="variants.size")
        ],
        values_list=["*match_params.variant_options"]
    )

# Create a value mapper instance
value_mapper = NestedValueMapper()

# Define match parameters
match_params = {
    "min_rating": 4,
    "variant_options": ["red", "large"]
}

# Resolve values for each mapping
for mapping_name in ["reviews", "variants"]:
    attr_field_value = value_mapper.get_field_value(mapping_name)
    print(f"{mapping_name}:")
    print(f"  Fields: {attr_field_value.fields}")
    print(f"  Values List: {attr_field_value.values_list}")
    print(f"  Values Map: {attr_field_value.values_map}")
    print()

This example demonstrates how to use nested fields in value mappers:

1. `reviews`: Maps to a nested field "reviews.rating" with a value from `match_params.min_rating`
2. `variants`: Maps to multiple nested fields ("variants.color" and "variants.size") with values from `match_params.variant_options`

When these mappings are used with directives, the toolkit automatically generates the appropriate nested queries.

## Extending Value Mappers

Value mappers can be extended to create more specialized mappers:

In [ ]:
class BaseProductValueMapper(DirectiveValueMapper):
    # Common mappings
    category = FieldValue(
        fields=["category"],
        values_list=["match_params.category"]
    )
    in_stock = FieldValue(
        fields=["in_stock"],
        values_list=[True]
    )

class ElectronicsValueMapper(BaseProductValueMapper):
    # Electronics-specific mappings
    brand = FieldValue(
        fields=["brand"],
        values_list=["match_params.brand"]
    )
    features = FieldValue(
        fields=["features"],
        values_list=["*match_params.features"]
    )
    
    # Override category mapping
    category = FieldValue(
        fields=["category", "subcategory"],
        values_list=["electronics"]
    )

# Create value mapper instances
base_mapper = BaseProductValueMapper()
electronics_mapper = ElectronicsValueMapper()

# Compare category mapping
print("Base Mapper - category:")
attr_field_value = base_mapper.get_field_value("category")
print(f"  Fields: {attr_field_value.fields}")
print(f"  Values List: {attr_field_value.values_list}")
print()

print("Electronics Mapper - category:")
attr_field_value = electronics_mapper.get_field_value("category")
print(f"  Fields: {attr_field_value.fields}")
print(f"  Values List: {attr_field_value.values_list}")
print()

# Check electronics-specific mapping
print("Electronics Mapper - brand:")
attr_field_value = electronics_mapper.get_field_value("brand")
print(f"  Fields: {attr_field_value.fields}")
print(f"  Values List: {attr_field_value.values_list}")

This example demonstrates how to extend value mappers:

1. We define a `BaseProductValueMapper` with common mappings
2. We extend it with an `ElectronicsValueMapper` that adds electronics-specific mappings
3. We override the `category` mapping in the `ElectronicsValueMapper`

This approach allows you to create a hierarchy of value mappers for different domains while reusing common mappings.

## Summary

In this notebook, we've explored value mapping in depth. We've covered:

- Creating basic value mappers
- Using value mappers with engines
- Dynamic value resolution
- Working with nested fields
- Extending value mappers

Value mapping is a powerful feature of the Elasticsearch Query Toolkit that allows you to define relationships between directive names, fields, and values in a structured, reusable way. It simplifies query construction and makes your code more maintainable.

In the next notebook, we'll explore boolean directives and how to combine multiple match conditions using boolean logic.